In [1]:
import pandas as pd
import re
import datetime

#from dotenv import load_dotenv
import requests
import os
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

#NLTK
import nltk
from nltk import word_tokenize, pos_tag

%matplotlib inline
#load_dotenv()

In [9]:
ratings = pd.read_csv('./input/ratings_small.csv')
metadata = pd.read_csv('./input/movies_metadata.csv', low_memory=False)

In [33]:
#ratings_full = pd.read_csv('./input/ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [6]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [7]:
ratings.shape

(100004, 4)

In [13]:
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# Grouping by userid

In [29]:
ratings_num = ratings_full.groupby('movieId').count()

In [30]:
ratings_num.head()

,userId,rating,timestamp
movieId,,,
1,247,247,247
2,107,107,107
3,59,59,59
4,13,13,13
5,56,56,56


In [15]:
ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [16]:
# returns datetime objects
#df['Timestamp'] = df.apply(lambda row: dt.datetime(row.year, row.month, row.day, row.hour), axis=1)
# Create a DatetimeIndex and assign it to the dataframe.
#ratings.index = pd.DatetimeIndex(ratings.date)

In [32]:
ratings_num.describe()

,userId,rating,timestamp
count,9066.000000,9066.000000,9066.000000
mean,11.030664,11.030664,11.030664
std,24.050800,24.050800,24.050800
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,3.000000,3.000000,3.000000
75%,9.000000,9.000000,9.000000
max,341.000000,341.000000,341.000000


# Users Clustering by movies keywords

In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [10]:
movies = ratings['movieId'].unique()

In [11]:
ratings['movieId'].value_counts().head()

356    341
296    324
318    311
593    304
260    291
Name: movieId, dtype: int64

In [18]:
def processOverview(overview):
    valid_tags = ["JJ","JJR","JJS","NN","NNS"]
    tokens = [item for item, tag in pos_tag(word_tokenize(overview)) if tag in valid_tags]
    return tokens

metadata['keywords'] = metadata['overview'].apply(lambda x: processOverview(str(x)))

In [46]:
keywords = []
for movie in metadata['keywords']:
    for word in movie:
        keywords.append(word)
    

In [48]:
len(keywords)

770605